# <span style="color:#36454F">Part 2: Problem Statement and Dataset</span>

## <span style="color:#5F9EA0">Topic of Interest:</span>
<span style="font-size: 16px">Analysis of Workplace Injuries in the Construction Sector</span>

## <span style="color:#36454F">Problem Statement</span>

1. Regression\
**S** - Model will predict monthly/yearly workplace accidents/injuries (by individual)\
**M** - Performance to be guided by RMSE\
**A** - Time-based so not sure if the features are suitable to predict rates as this is based on accident data (i.e. accident has already occurred). Using this data to predict number of accidents in a time frame does not seem feasible\
**R** - Show workplace injuries trends and the correlated factors\
**T** - <span style="color:#E56E94">Idea Aborted</span>


2. Classification - Creating a risk profile\
**S** - Model will predict degree of injury\
**M** - Performance to be guided by accuracy and F1 score\
**A** - If by accident, features would be from one-hot encoded event keywords. If by individual injuries, human, work nature, env features can be used as variables.\
**R** - Risk profile to show what factors could lead to serious injury, areas to focus on safety training or increase in inspections for certain work.\
**T** - 🕛🕒🕕🕘🕛

<span style="color:#5F9EA0">I will build a classification model to predict the degree of injuries from the accident dataset over the last 5/10 years and identify the correlated features that determine the rankings. Accuracy and F1 score would be used as success metrics.</span>


SMART: Specific, Measurable, Achievable, Relevant, Time-bound

## <span style="color:#36454F">Dataset Source(s):</span>
Occupational Safety and Health Administration (OSHA) [data](https://enforcedata.dol.gov/views/data_summary.php) uploaded by the US Department of Labor.
- Time period: 1970 - present (Retrieved February 17, 2022)


## <span style="color:#36454F">Import Libraries</span>

In [104]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
pd.set_option("display.max_colwidth", None)

## <span style="color:#36454F">Read Files</span>

In [2]:
accident_df = pd.read_csv('osha_accident.csv')
accident_abstract_df = pd.read_csv('osha_accident_abstract.csv')
accident_injury_df = pd.read_csv('osha_accident_injury.csv')
accident_df.shape, accident_abstract_df.shape, accident_injury_df.shape

((142212, 16), (206784, 21), (992789, 4))

***Note:*** The 3 files have different shapes. The accident abstract file was downloaded on 18 Feb but it does not explain the huge difference in rows. This is a possible indication of duplicate rows.

In [119]:
inspect_files = glob.glob('data/osha_inspection_20220217/osha_inspection*.csv')
inspect_files

['data/osha_inspection_20220217/osha_inspection0.csv',
 'data/osha_inspection_20220217/osha_inspection1.csv',
 'data/osha_inspection_20220217/osha_inspection3.csv',
 'data/osha_inspection_20220217/osha_inspection2.csv',
 'data/osha_inspection_20220217/osha_inspection4.csv']

In [120]:
inspection_df = pd.concat((pd.read_csv(f) for f in inspect_files), ignore_index=True)
inspection_df.shape

/var/folders/pf/rrh9p6sx2dd51926v2m3hf9m0000gn/T/ipykernel_15461/2493664863.py:1: DtypeWarning: Columns (8,16,20,23,24,25,26,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  inspection_df = pd.concat((pd.read_csv(f) for f in inspect_files), ignore_index=True)
/var/folders/pf/rrh9p6sx2dd51926v2m3hf9m0000gn/T/ipykernel_15461/2493664863.py:1: DtypeWarning: Columns (18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  inspection_df = pd.concat((pd.read_csv(f) for f in inspect_files), ignore_index=True)
/var/folders/pf/rrh9p6sx2dd51926v2m3hf9m0000gn/T/ipykernel_15461/2493664863.py:1: DtypeWarning: Columns (20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  inspection_df = pd.concat((pd.read_csv(f) for f in inspect_files), ignore_index=True)
/var/folders/pf/rrh9p6sx2dd51926v2m3hf9m0000gn/T/ipykernel_15461/2493664863.py:1: DtypeWarning: Columns (16,18,20,22,23,24,25,26,27,29) have

(4866341, 36)

## <span style="color:#36454F">Data Types & Missing Values</span>

In [12]:
accident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142212 entries, 0 to 142211
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   summary_nr     142212 non-null  int64  
 1   report_id      142212 non-null  int64  
 2   event_date     142212 non-null  object 
 3   event_time     0 non-null       float64
 4   event_desc     142210 non-null  object 
 5   event_keyword  141944 non-null  object 
 6   const_end_use  30619 non-null   object 
 7   build_stories  21452 non-null   float64
 8   nonbuild_ht    15276 non-null   float64
 9   project_cost   25494 non-null   object 
 10  project_type   30985 non-null   object 
 11  sic_list       109089 non-null  object 
 12  fatality       64306 non-null   object 
 13  state_flag     0 non-null       float64
 14  abstract_text  0 non-null       float64
 15  load_dt        142212 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 17.4+ MB


***Note:*** The time of event, accident abstract and state details are empty!

In [14]:
accident_abstract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992789 entries, 0 to 992788
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   summary_nr     992789 non-null  int64 
 1   line_nr        992789 non-null  int64 
 2   abstract_text  992228 non-null  object
 3   load_dt        992789 non-null  object
dtypes: int64(2), object(2)
memory usage: 30.3+ MB


In [13]:
accident_injury_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206784 entries, 0 to 206783
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   summary_nr      206784 non-null  int64  
 1   rel_insp_nr     206784 non-null  int64  
 2   age             206784 non-null  int64  
 3   sex             133428 non-null  object 
 4   nature_of_inj   172221 non-null  float64
 5   part_of_body    172221 non-null  float64
 6   src_of_injury   172221 non-null  float64
 7   event_type      206701 non-null  float64
 8   evn_factor      172216 non-null  float64
 9   hum_factor      172218 non-null  float64
 10  occ_code        202753 non-null  float64
 11  degree_of_inj   206702 non-null  float64
 12  task_assigned   203264 non-null  float64
 13  hazsub          29287 non-null   object 
 14  const_op        171183 non-null  float64
 15  const_op_cause  178205 non-null  float64
 16  fat_cause       171090 non-null  float64
 17  fall_dista

## <span style="color:#36454F">Checks for Duplicates</span>

In [70]:
print('accident_df: ', accident_df.shape, accident_df['summary_nr'].unique().shape)
print('accident_abstract_df: ', accident_abstract_df.shape, accident_abstract_df['summary_nr'].unique().shape)
print('accident_injury_df: ', accident_injury_df.shape, accident_injury_df['summary_nr'].unique().shape)

accident_df:  (142212, 16) (142212,)
accident_abstract_df:  (992789, 4) (142235,)
accident_injury_df:  (206784, 21) (164716,)


The accident df has no duplicate accident form IDs but the abstract and injury dataframes have duplicate cases.

### <span style="color:#045F5F">Compare accident_df & accident_abstract_df</span>

In [71]:
#Merge accident_df on accident_abstract_df to identify "missing" accidents
df_all_abstract = accident_abstract_df.merge(accident_df.drop_duplicates(), on=['summary_nr'], 
                                             how='left', indicator=True)
print(df_all_abstract.shape)
df_all_abstract['_merge'].value_counts()

(992789, 20)


both          992592
left_only        197
right_only         0
Name: _merge, dtype: int64

In [74]:
#Missing accident abstracts
df_all_abstract['load_dt_x'].loc[df_all_abstract['_merge']=='left_only'].unique()

array(['2022-02-18 00:20:15 EST'], dtype=object)

***Observation:*** The additional 197 records were due to the 1 day lag in retrieving the dataset. The other accident form IDs (142k unique) managed to match with 992k in the abstract dataframe; indicating many duplicates as seen below - Form 202614442 had 110 rows! 

In [190]:
df_all_abstract['summary_nr'].value_counts().head()

202614442    110
202588034     98
201634052     94
201780053     83
202713764     82
Name: summary_nr, dtype: int64

In [92]:
#Examine form with most rows in accident_abstract_df
df_all_abstract.loc[df_all_abstract['summary_nr']==202614442]

,summary_nr,line_nr,abstract_text_x,load_dt_x,report_id,event_date,event_time,event_desc,event_keyword,const_end_use,...,nonbuild_ht,project_cost,project_type,sic_list,fatality,state_flag,abstract_text_y,load_dt_y,_merge,char_count
782684,202614442,1,"""The receiving dock attendant and nine other adjacently exposed workers were sen",2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782685,202614442,2,t to emergency rooms for evaluation and were released with only minor injuries.,2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782686,202614442,3,"All others employees were unharmed."" Employee #1 was the truck driver. He was wo",2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782687,202614442,4,"rking for Univar USA. In addition to Employee #1, who was employed by Univar USA",2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782688,202614442,5,", ten other workers were injured in this incident. They were employees of Darigo",2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782789,202614442,106,usy to double check the delivery.) Positive -The tanker truck was properly label,2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782790,202614442,107,ed. -The pipe hook-ups in the wall of the load dock were properly labeled and co,2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782791,202614442,108,lor coded. -The pipe hook-ups were locked. However the lock combinations were gi,2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,DOCK WORKER,TOXIC FUMES,TRUCK DRIVER",NaN,...,NaN,NaN,NaN,2026,NaN,NaN,NaN,2019-07-22 00:18:31 EDT,both,80.0
782792,202614442,109,ven to the truck driver. -Emergency evacuation was successful. -The CIP was unma,2019-07-22 00:19:35 EDT,1054191.0,2012-05-30 10:15:00,NaN,Truck Driver Sustains Burns from Inhaled Chemicals,"TANK TRUCK,SODIUM HYDROXIDE,SODIUM HYPOCHLORITE,INHALATION,CHLORINE,CHEMICAL REACTION,CHEMICAL BURN,D

In [191]:
# Number of characters
df_all_abstract['char_count'] = df_all_abstract['abstract_text_x'].str.len() ## this also includes spaces
df_all_abstract['char_count'].loc[df_all_abstract['summary_nr']==202614442].mode()

0    80.0
Name: char_count, dtype: float64

***Note:*** The reason for duplicates was due to a 80-character limit in the abstract_text column. We would need to append the rows by summary_nr if a more detailed examination into the abstract is neccessary. Otherwise, we can proceed with the "event_keyword" first.

### <span style="color:#045F5F">Compare accident_df & accident_injury_df</span>

In [213]:
#Merge accident_df on accident_injury_df to identify "missing" accidents
df_all_injury = accident_injury_df.merge(accident_df.drop_duplicates(), on=['summary_nr'], how='left', indicator=True)
print(df_all_injury.shape)
df_all_injury['_merge'].value_counts()

(206784, 38)


both          172606
left_only      34178
right_only         0
Name: _merge, dtype: int64

In [214]:
#Missing accidents in accident_df
missing_acc_forms = df_all_injury[['summary_nr','rel_insp_nr','sic_list']].loc[df_all_injury['_merge']=='left_only']
missing_acc_forms.reset_index(inplace=True,drop=True) #reset index in place and without adding new column of old index

***Assumption:*** column "rel_insp_nr" in accident_injury_df is the same as "activity_nr" in inspection_df

|Dataframe|Column|Data Type|Description|
|---|---|---|---|
|accident_injury_df|summary_nr|numeric(9,0)|Identifies the accident OSHA-170 form|
|accident_injury_df|rel_insp_nr|numeric(9,0)|Identifies the inspection associated with investigating a particular injury|
|inspection_df|activity_nr|numeric(9,0)|Unique identifier for the inspection|

In [215]:
#List of interested columns from inspection_df
attri_list = ['activity_nr','insp_type','open_date','case_mod_date','close_conf_date','close_case_date','sic_code']

In [216]:
#Left merge on form IDs that are missing from accident_df but available in accident_injury_df
missing_acc_forms = missing_acc_forms.merge(inspection_df[attri_list], left_on='rel_insp_nr', 
                                            right_on='activity_nr', how='left')
# missing_acc_forms

In [217]:
#Year when inspection opened
missing_acc_forms['year_open'] = missing_acc_forms['open_date'].str[:4]
missing_acc_forms['year_open'] = missing_acc_forms['year_open'].astype(float)

#Year when inspection closed
missing_acc_forms['year_close'] = missing_acc_forms['close_case_date'].str[:4]
missing_acc_forms['year_close'] = missing_acc_forms['year_close'].astype(float)

#Year when accident occurred
accident_df['event_year'] = accident_df['event_date'].str[:4]
accident_df['event_year'] = accident_df['event_year'].astype(int)

In [219]:
# missing_acc_forms[['year_open','year_close']].describe().loc[['count','mean','min','max']]
missing_acc_forms[['year_open','year_close']].describe()

,year_open,year_close
count,34158.000000,33987.000000
mean,1981.841560,1983.698444
std,9.191668,8.965512
min,1973.000000,1974.000000
25%,1977.000000,1980.000000
50%,1980.000000,1982.000000
75%,1982.000000,1984.000000
max,2015.000000,2019.000000


In [202]:
missing_acc_forms['summary_nr'].unique().shape

(22509,)

In [220]:
accident_df['event_year'].describe()

count    142212.000000
mean       2003.742778
std          10.301551
min        1972.000000
25%        1995.000000
50%        2004.000000
75%        2012.000000
max        2021.000000
Name: event_year, dtype: float64

In [232]:
print("Missing data percentage (%): ",round(((missing_acc_forms.loc[missing_acc_forms['year_open']>2000].shape[0])
                                            /(accident_df.loc[accident_df['event_year']>2000].shape[0] + 
                                              missing_acc_forms.loc[missing_acc_forms['year_open']>2000].shape[0])*100),2))

Missing data percentage (%):  2.79


***Observation:*** 22k unique accident forms missing from accident_df but majority were prior to 2000s. The missing rows could thus be treated as negligible as they make up less than 5% of the dataset (including duplicates).

In [89]:
df_all_injury['summary_nr'].value_counts()

201921863    152
201923414    104
170073035     87
118174        86
14303978      75
            ... 
170739627      1
170739635      1
170739643      1
170739650      1
980213516      1
Name: summary_nr, Length: 164716, dtype: int64

In [290]:
df_all_injury[['rel_insp_nr','age','sex','fatality','event_keyword']].loc[df_all_injury['summary_nr']==201921863]

,rel_insp_nr,age,sex,fatality,event_keyword
148627,123578494,30,M,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148628,123578528,0,NaN,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148629,123578544,25,M,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148630,123578544,29,F,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148631,123578551,0,NaN,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
...,...,...,...,...,...
148774,301406500,35,M,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148775,301407409,0,NaN,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148776,301407417,0,NaN,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"
148777,301407425,0,NaN,X,"EXPLOSION,FIRE,BURN,HEAT,SPRAIN,CHEMICAL,CHEMICAL BURN,CHEMICAL REACTION,CHEMICAL VAPOR"


In [237]:
df_all_injury.iloc[148628].T

summary_nr                                                                                      201921863
rel_insp_nr                                                                                     123578528
age                                                                                                     0
sex                                                                                                   NaN
nature_of_inj                                                                                         0.0
part_of_body                                                                                          0.0
src_of_injury                                                                                         0.0
event_type                                                                                            0.0
evn_factor                                                                                            0.0
hum_factor                                    

***Notes:*** Reason for duplicates in accident forms in accident_injury_df was due to the number of injuries incurred. E.g. accident form ID 201921863 had 1 fatality and 145 injured. This explains 146 of the 152 duplicates but there are still some extra entries.

## <span style="color:#36454F">Subsetting the dataframe</span>

The construction related variable which returned the highest number of cases was "const_op_cause" from the accident_injury_df. This showed that there were many missing values in the construction variables in the accident_df.

In [296]:
construction_df = df_all_injury.loc[df_all_injury['const_op_cause'].notnull()==True]
construction_df['summary_nr'].unique().shape

(138715,)

In [297]:
construction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178205 entries, 0 to 206783
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   summary_nr      178205 non-null  int64   
 1   rel_insp_nr     178205 non-null  int64   
 2   age             178205 non-null  int64   
 3   sex             133428 non-null  object  
 4   nature_of_inj   170715 non-null  float64 
 5   part_of_body    170717 non-null  float64 
 6   src_of_injury   170717 non-null  float64 
 7   event_type      178200 non-null  float64 
 8   evn_factor      170717 non-null  float64 
 9   hum_factor      170716 non-null  float64 
 10  occ_code        177608 non-null  float64 
 11  degree_of_inj   178200 non-null  float64 
 12  task_assigned   177453 non-null  float64 
 13  hazsub          29145 non-null   object  
 14  const_op        170718 non-null  float64 
 15  const_op_cause  178205 non-null  float64 
 16  fat_cause       170687 non-null  float